# Clifford+RZ

You need the `pyqrack` package to run this notebook. [`vm6502q/pyqrack`](https://github.com/vm6502q/pyqrack) is a pure Python wrapper on the [`vm6502q/qrack`](https://github.com/vm6502q/qrack) quantum computer simulation framework core library. The preferred method of installation is from source code, at those GitHub repositories, but a package with default build precompiled binaries is available on [pypi](https://pypi.org/project/pyqrack/0.2.0/).

In [1]:
# For example, if your Jupyter installation uses pip:
# import sys
# !{sys.executable} -m pip install --platform=manylinux2014_x86_64 --only-binary=:all: pyqrack

In [2]:
import collections
import copy
import math
import os
import random

import numpy as np

import tensorcircuit as tc
import tensorcircuit.compiler.simple_compiler as tcsc

from qiskit import QuantumCircuit
from qiskit.compiler import transpile

from pyqrack import QrackSimulator

# If we disable OpenCL and set the max CPU qubits to max integer, we bypass qubit widths limits.
os.environ["QRACK_MAX_CPU_QB"]="-1"

In [3]:
width = 4
max_magic = 4

[`QrackSimulator`](https://github.com/vm6502q/pyqrack/blob/main/pyqrack/qrack_simulator.py) is the "workhorse" of the `pyqrack` package. It instantiates simulated "registers" of qubits that we can act basic quantum gates between, to form arbitrary universal quantum circuits.

`QrackCircuit` is an optional class for optimizing compilation. With it, one can define a circuit in advance, which is optimized upon definition. Then, one can save the optimized result to a file and later load it into a new or existing `QrackCircuit` instance. Ultimately, the circuit is executed by calling `run()` on a `QrackCircuit`, with a parameter of `QrackSimulator` of appropriate size. (The necessary width of the `QrackSimulator` can be determined with `QrackCircuit.get_qubit_count()`.)

In [4]:
def rand_1qb(sim, q):
    # th = random.uniform(0, 4 * math.pi)
    ph = random.uniform(0, 4 * math.pi)
    # lm = random.uniform(0, 4 * math.pi)
    # sim.u(th, ph, lm, q)
    sim.u(0, ph, 0, q)

def random_circuit(width, circ):
    t_count = 0
    single_bit_gates = circ.h, circ.x, circ.y
    two_bit_gates = circ.cx, circ.cy, circ.cz, circ.swap, circ.iswap

    for n in range(3 * width):
        # Single bit gates
        for j in range(width):
            random.choice(single_bit_gates)(j)
            if (t_count < max_magic) and ((width * width * random.random() / max_magic) < 1):
                rand_1qb(circ, j)
                t_count += 1

        # Multi bit gates
        bit_set = [i for i in range(width)]
        while len(bit_set) > 1:
            b1 = random.choice(bit_set)
            bit_set.remove(b1)
            b2 = random.choice(bit_set)
            bit_set.remove(b2)
            random.choice(two_bit_gates)(b1, b2)

    return circ

If we have Qiskit and numpy installed, we can convert to a Qiskit circuit. The gate count and qubit width might be higher, but the circuit is entirely Clifford group, except for a single terminal layer of non-Clifford gates, (followed by either post selection or a rudimentary error-correction routine).

In [5]:
orig_circ = random_circuit(width, QuantumCircuit(width))
qsim = QrackSimulator(width, isSchmidtDecomposeMulti=False, isSchmidtDecompose=False, isOpenCL=False)
qsim.run_qiskit_circuit(orig_circ, 0)
qsim.out_to_file("qrack_circuit.chp")
qsim.reset_all()
qsim = None

In [6]:
circ = QrackSimulator.file_to_qiskit_circuit("qrack_circuit.chp")

if circ.width() <= 16:
    print(circ)
print("(Near-Clifford) gate count: ", sum(dict(circ.count_ops()).values()))
print("(Near-Clifford) depth of critical path: ", circ.depth())

     ┌───┐┌───┐   ┌───┐┌───┐┌───┐           ┌───┐          ┌───┐┌───┐»
q_0: ┤ H ├┤ S ├─X─┤ S ├┤ H ├┤ S ├─────■─────┤ X ├──■───────┤ X ├┤ Z ├»
     ├───┤└───┘ │ ├───┤├───┤└───┘     │     └─┬─┘┌─┴─┐┌───┐└─┬─┘└───┘»
q_1: ┤ H ├──────┼─┤ X ├┤ S ├──────────┼───────┼──┤ X ├┤ H ├──■────■──»
     ├───┤      │ └─┬─┘└───┘        ┌─┴─┐     │  └───┘└───┘     ┌─┴─┐»
q_2: ┤ H ├──────┼───┼────■──────────┤ X ├─────┼─────────────────┤ X ├»
     └───┘      │   │  ┌─┴─┐┌───┐┌──┴───┴──┐  │                 └───┘»
q_3: ───────────X───■──┤ X ├┤ Z ├┤ Unitary ├──┼──────────────────────»
     ┌───┐             └───┘└───┘└─────────┘  │  ┌───┐               »
q_4: ┤ H ├────────────────────────────────────■──┤ H ├───────────────»
     └───┘                                       └───┘               »
«     ┌─────────┐                                
«q_0: ┤ Unitary ├────────────────────────────────
«     └─────────┘┌───┐┌───┐┌─────────┐           
«q_1: ───────────┤ X ├┤ Y ├┤ Unitary ├───────────
«        ┌───┐   └─

We can further optimize by post-selecting with tensor network software.

In [7]:
tc.set_backend("tensorflow")
tc.set_contractor("auto")
tc.set_dtype("complex64")

net = tc.Circuit.from_qiskit(circ)
for b in range(width, circ.width()):
    net.post_select(b, keep=0)
net = tcsc.simple_compile(net)[0]

We can sample with tensorcircuit at this point, or we can try another round of optimization.

In [8]:
# %%time
# print(net.sample(allow_state=True, batch=1024, format="count_dict_bin"))

We can convert back to a Qiskit circuit.

In [9]:
circ = net.to_qiskit()
if circ.width() <= 16:
    print(circ)
print("(Near-Clifford) gate count: ", sum(dict(circ.count_ops()).values()))
print("(Near-Clifford) depth of critical path: ", circ.depth())

omit non unitary gate in tensorcircuit when transforming to qiskit: any


     ┌───┐┌───┐   ┌───┐┌───┐┌───┐     ┌───┐          ┌───┐┌───┐               »
q_0: ┤ H ├┤ S ├─X─┤ S ├┤ H ├┤ S ├──■──┤ X ├──■───────┤ X ├┤ Z ├───────────────»
     ├───┤└───┘ │ ├───┤├───┤└───┘  │  └─┬─┘┌─┴─┐┌───┐└─┬─┘└───┘     ┌───┐┌───┐»
q_1: ┤ H ├──────┼─┤ X ├┤ S ├───────┼────┼──┤ X ├┤ H ├──■────■───────┤ X ├┤ Y ├»
     ├───┤      │ └─┬─┘└───┘     ┌─┴─┐  │  └───┘└───┘     ┌─┴─┐┌───┐└─┬─┘└───┘»
q_2: ┤ H ├──────┼───┼────■───────┤ X ├──┼─────────────────┤ X ├┤ H ├──■────■──»
     └───┘      │   │  ┌─┴─┐┌───┐└───┘  │                 └───┘└───┘       │  »
q_3: ───────────X───■──┤ X ├┤ Z ├───────┼──────────────────────────────────┼──»
     ┌───┐             └───┘└───┘       │  ┌───┐                         ┌─┴─┐»
q_4: ┤ H ├──────────────────────────────■──┤ H ├─────────────────────────┤ X ├»
     └───┘                                 └───┘                         └───┘»
«                 
«q_0: ────────────
«                 
«q_1: ────────────
«                 
«q_2: ────────────
«     

Some phase gates tend to occur at the beginning of the circuit: these can safely be ignored. In fact, we can trace out the evolution of separable single-qubit states from the beginning of the circuit until they become entangled, for more gate elimination. Deterministic control qubits can be eliminated, as well.

In [10]:
def qunit_step(i, circ, to_cut, state, basis, has_depth):
    op = circ.data[i].operation
    qubits = circ.data[i].qubits
    first_qubit = circ.find_bit(qubits[0])[0]
    if has_depth[first_qubit]:
        for q in qubits[1:]:
            has_depth[circ.find_bit(q)[0]] = True
        return
    if len(qubits) == 1:
        if op.name == "h":
            basis[first_qubit] = not basis[first_qubit]
        elif ((not basis[first_qubit]) and ((op.name == "z") or (op.name == "s") or (op.name == "sdg"))) or (basis[first_qubit] and (op.name == "x")):
            to_cut.append(i)
        elif ((not basis[first_qubit]) and ((op.name == "x") or (op.name == "y"))) or (basis[first_qubit] and ((op.name == "y") or (op.name == "z"))):
            state[first_qubit] = not state[first_qubit]
        else:
            has_depth[first_qubit] = True
    elif op.name == "swap":
        second_qubit = circ.find_bit(qubits[1])[0]
        has_depth[first_qubit], has_depth[second_qubit] = has_depth[second_qubit], has_depth[first_qubit]
        state[first_qubit], state[second_qubit] = state[second_qubit], state[first_qubit]
        basis[first_qubit], basis[second_qubit] = basis[second_qubit], basis[first_qubit]
        if (not has_depth[first_qubit]) and (not has_depth[second_qubit]) and (state[first_qubit] == state[second_qubit]) and (basis[first_qubit] == basis[second_qubit]):
            to_cut.append(i)
    elif not basis[first_qubit] and op.name == "cx":
        if state[first_qubit]:
            circ.data[i].operation.name = "x"
            del circ.data[i].qubits[0]
            i -= 1
        else:
            to_cut.append(i)
    else:
        qc = circ.find_bit(qubits[0])[0]
        qt = circ.find_bit(qubits[1])[0]
        has_depth[qc] = True
        has_depth[qt] = True

to_cut = []
state = circ.width() * [False]
basis = circ.width() * [False]
has_depth = circ.width() * [False]
for i in range(len(circ.data)):
    qunit_step(i, circ, to_cut, state, basis, has_depth)
to_cut.reverse()
for i in to_cut:
    del circ.data[i]

to_cut = []
state = circ.width() * [False]
basis = circ.width() * [False]
has_depth = (width * [True]) + ((circ.width() - width) * [False])
for i in reversed(range(len(circ.data))):
    qunit_step(i, circ, to_cut, state, basis, has_depth)
for i in to_cut:
   del circ.data[i]

if circ.width() <= 16:
    print(circ)
print("(Near-Clifford) gate count: ", sum(dict(circ.count_ops()).values()))
print("(Near-Clifford) depth of critical path: ", circ.depth())

     ┌───┐┌───┐   ┌───┐┌───┐     ┌───┐          ┌───┐┌───┐                    »
q_0: ┤ H ├┤ S ├─X─┤ H ├┤ S ├──■──┤ X ├──■───────┤ X ├┤ Z ├────────────────────»
     ├───┤└───┘ │ ├───┤├───┤  │  └─┬─┘┌─┴─┐┌───┐└─┬─┘└───┘     ┌───┐┌───┐     »
q_1: ┤ H ├──────┼─┤ X ├┤ S ├──┼────┼──┤ X ├┤ H ├──■────■───────┤ X ├┤ Y ├─────»
     ├───┤      │ └─┬─┘└───┘┌─┴─┐  │  └───┘└───┘     ┌─┴─┐┌───┐└─┬─┘└───┘     »
q_2: ┤ H ├──────┼───┼────■──┤ X ├──┼─────────────────┤ X ├┤ H ├──■────■───────»
     └───┘      │   │  ┌─┴─┐├───┤  │                 └───┘└───┘       │       »
q_3: ───────────X───■──┤ X ├┤ Z ├──┼──────────────────────────────────┼───────»
     ┌───┐             └───┘└───┘  │  ┌───┐                         ┌─┴─┐┌───┐»
q_4: ┤ H ├─────────────────────────■──┤ H ├─────────────────────────┤ X ├┤ X ├»
     └───┘                            └───┘                         └───┘└───┘»
«            
«q_0: ───────
«            
«q_1: ───────
«            
«q_2: ───────
«            
«q_3: ───────
«     ┌─

If any ancilla qubit is nothing but an injection gadget, we can re-inject it.

In [11]:
sqrt1_2 = 1 / math.sqrt(2)
passable_gates = ["unitary", "h", "x", "y", "z", "s", "sdg"]

# is_injectable = False
# for i in range(width, circ.width()):
#     is_injectable = True
#     for j in reversed(range(len(circ.data))):
#         op = circ.data[j].operation
#         qubits = circ.data[j].qubits

#         id = circ.find_bit(qubits[0])[0]
#         if len(qubits) == 1 and id == i and not (op.name in passable_gates):
#             is_injectable = False
#             break

#         if len(qubits) < 2:
#            continue

#         if (id == i) or not (op.name == "cx"):
#             is_injectable = False
#            break

if True:
    for i in range(width, circ.width()):
        non_clifford = np.array([[sqrt1_2, sqrt1_2], [sqrt1_2, -sqrt1_2]], np.complex128)
        to_cut = []
        for j in reversed(range(len(circ.data))):
            op = circ.data[j].operation
            qubits = circ.data[j].qubits

            id = circ.find_bit(qubits[0])[0]
            if (len(qubits) < 2) and (id == i):
                to_cut.append(j)
                if op.name == "unitary":
                    non_clifford = np.matmul(non_clifford, op.params[0])
                elif op.name == "h":
                    non_clifford = np.matmul(non_clifford, np.array([[sqrt1_2, sqrt1_2], [sqrt1_2, -sqrt1_2]]))
                elif op.name == "x":
                    non_clifford = np.matmul(non_clifford, np.array([[0, 1], [1, 0]]))
                elif op.name == "y":
                    non_clifford = np.matmul(non_clifford, np.array([[0, -1j], [1j, 0]]))
                elif op.name == "z":
                    non_clifford = np.matmul(non_clifford, np.array([[1, 0], [0, -1]]))
                elif op.name == "s":
                    non_clifford = np.matmul(non_clifford, np.array([[1, 0], [0, 1j]]))
                elif op.name == "sdg":
                    non_clifford = np.matmul(non_clifford, np.array([[1, 0], [0, -1j]]))
                else: 
                    print("Something went wrong!")

                continue

            if len(qubits) < 2:
                continue

            if (id == i) or not (op.name == "cx"):
                # non_clifford = np.matmul(non_clifford, np.array([[sqrt1_2, sqrt1_2], [sqrt1_2, -sqrt1_2]]))
                c = QuantumCircuit(1)
                c.unitary([[1, 0], [0, 1]], 0)
                instr = c.data[0]
                instr.qubits = (qubits[0],)
                instr.operation.params=[non_clifford]
                circ.data.insert(j + 1, copy.deepcopy(instr))
                break

            if (circ.find_bit(qubits[1])[0] == i) and (op.name == "cx"):
                c = QuantumCircuit(1)
                c.unitary([[1, 0], [0, 1]], 0)
                instr = c.data[0]
                instr.qubits = (qubits[0],)
                instr.operation.params=[non_clifford]
                circ.data[j] = copy.deepcopy(instr)
        
        for i in to_cut:
            del circ.data[i]

if circ.width() <= 16:
    print(circ)
print("(Near-Clifford) gate count: ", sum(dict(circ.count_ops()).values()))
print("(Near-Clifford) depth of critical path: ", circ.depth())

     ┌───┐┌───┐   ┌───┐┌───┐     ┌───┐                 ┌───┐ ┌───┐          
q_0: ┤ H ├┤ S ├─X─┤ H ├┤ S ├──■──┤ X ├─────■───────────┤ X ├─┤ Z ├──────────
     ├───┤└───┘ │ ├───┤├───┤  │  └─┬─┘   ┌─┴─┐   ┌───┐ └─┬─┘ └───┘     ┌───┐
q_1: ┤ H ├──────┼─┤ X ├┤ S ├──┼────┼─────┤ X ├───┤ H ├───■─────■───────┤ X ├
     ├───┤      │ └─┬─┘└───┘┌─┴─┐  │     └───┘   └───┘       ┌─┴─┐┌───┐└─┬─┘
q_2: ┤ H ├──────┼───┼────■──┤ X ├──┼─────────────────────────┤ X ├┤ H ├──■──
     └───┘      │   │  ┌─┴─┐├───┤  │                         └───┘└───┘     
q_3: ───────────X───■──┤ X ├┤ Z ├──┼────────────────────────────────────────
     ┌───┐             └───┘└───┘  │  ┌─────────┐┌───┐┌─────┐               
q_4: ┤ H ├─────────────────────────■──┤ Unitary ├┤ H ├┤ any ├───────────────
     └───┘                            └─────────┘└───┘└─────┘               
(Near-Clifford) gate count:  24
(Near-Clifford) depth of critical path:  13


`QrackSimulator` can run a Qiskit circuit. (After the file is saved and loaded, separable ancillae qubits will be removed.)

In [12]:
basis_gates=["rz", "h", "x", "y", "z", "sx", "sy", "s", "sdg", "cx", "cy", "cz", "swap", "iswap"]
circ = transpile(circ, basis_gates=basis_gates, optimization_level=3)

os.rename("qrack_circuit.chp", "qrack_circuit_original.chp")

qsim = QrackSimulator(circ.width(), isSchmidtDecomposeMulti=False, isSchmidtDecompose=False, isOpenCL=False)
qsim.run_qiskit_circuit(circ, 0)

qsim.out_to_file("qrack_circuit.chp")

lines = []
with open("qrack_circuit.chp","r",encoding="utf-8") as file:
    lines = file.readlines()
lines[0] = str(width) + "\n"
with open("qrack_circuit.chp", "w", encoding="utf-8") as file:
    file.writelines(lines)

qsim = QrackSimulator.in_from_file("qrack_circuit.chp")
qsim.out_to_file("qrack_circuit.chp")

Device #0, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_Intel(R)_UHD_Graphics_[0x9bc4].ir
Device #1, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_NVIDIA_GeForce_RTX_3080_Laptop_GPU.ir


We can also sample with Qrack, as opposed to tensor circuits.

In [13]:
lines = []
with open("qrack_circuit.chp","r",encoding="utf-8") as file:
    lines = file.readlines()
chp_qb_count = int(lines[1])
is_clifford = True
for i in range(2 + 2 * chp_qb_count, len(lines)):
    if lines[i] != "(1,0) (0,0) (0,0) (1,0)\n":
        is_clifford = False
        break

shot_count = 100000
samples = []
if is_clifford:
    qsim = QrackSimulator.in_from_file("qrack_circuit.chp")

    %time
    samples = qsim.measure_shots(list(range(width)), shot_count)
 
    # print(samples)
    print(qsim.measure_shots(list(range(width)), 256))

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 7.15 µs
[6, 6, 5, 5, 14, 8, 13, 9, 15, 10, 8, 1, 13, 13, 6, 1, 5, 6, 14, 11, 9, 8, 8, 13, 1, 6, 11, 7, 8, 0, 13, 3, 12, 14, 1, 14, 1, 8, 4, 6, 8, 14, 11, 10, 11, 0, 8, 2, 0, 11, 0, 15, 1, 10, 4, 0, 1, 7, 9, 13, 14, 0, 5, 7, 10, 11, 11, 6, 7, 4, 1, 2, 15, 3, 10, 8, 10, 12, 13, 10, 12, 2, 12, 3, 5, 0, 5, 1, 3, 4, 3, 4, 1, 5, 6, 5, 13, 0, 2, 2, 12, 12, 15, 8, 5, 13, 11, 7, 12, 0, 0, 6, 11, 7, 13, 7, 1, 7, 12, 4, 1, 8, 5, 7, 15, 9, 10, 12, 14, 1, 2, 4, 15, 13, 13, 9, 7, 4, 10, 8, 15, 1, 12, 7, 10, 15, 0, 0, 14, 4, 0, 12, 4, 12, 11, 6, 13, 6, 12, 14, 7, 4, 5, 5, 4, 5, 7, 7, 8, 9, 8, 7, 8, 1, 3, 3, 0, 10, 2, 1, 8, 11, 6, 1, 6, 10, 13, 14, 14, 1, 9, 12, 5, 12, 15, 14, 10, 7, 13, 8, 7, 1, 2, 15, 5, 14, 2, 13, 6, 10, 6, 2, 3, 13, 11, 1, 10, 5, 13, 13, 13, 7, 7, 9, 13, 0, 1, 7, 4, 14, 9, 6, 8, 3, 3, 7, 10, 8, 5, 5, 2, 6, 9, 10, 0, 10, 11, 2, 4, 12, 3, 8, 15, 6, 8, 3]


In [14]:
def fidelities_from_measurement_results(measurement_list, qiskit_circuit, register_width, ideal_shots = 1024):
    
    perm_count = 1 << register_width
    
    sim = QrackSimulator(qubitCount=register_width, qiskitCircuit = qiskit_circuit)
    ideal_result = sim.measure_shots(list(range(register_width)), ideal_shots)
    ideal_result = dict(collections.Counter(ideal_result))
    for key, value in ideal_result.items():
        ideal_result[key] = value / ideal_shots

    # This is a logically-grouped batch of qubit measurement results, as a list of "permutations."
    t_histogram = collections.Counter(measurement_list)
    shot_count = sum(t_histogram.values())
    t_histogram = dict(t_histogram)

    histogram = {}
    for key, value in t_histogram.items():
        histogram[key] = value / shot_count
        
    fidelity = 0
    for qubit_permutation in histogram.keys():
        ideal_normalized_frequency = ideal_result[qubit_permutation] if qubit_permutation in ideal_result else 0
        normalized_frequency = histogram[qubit_permutation]
        fidelity += math.sqrt(ideal_normalized_frequency * normalized_frequency)
    fidelity *= fidelity
                               
    return fidelity

if is_clifford and width <= 16:
    print("Fidelity: ", fidelities_from_measurement_results(samples, orig_circ, width, shot_count))

Fidelity:  0.9207486411508493
